In [1]:
import sklearn
import numpy as np
import pandas as pd

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

# from ner_eval import collect_named_entities
# from ner_eval import compute_metrics
# from ner_eval import compute_precision_recall_wrapper

# from sklearn_crfsuite import metric
from nervaluate import Evaluator

In [2]:
model_name='indolem/indobert-base-uncased'

In [3]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [4]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [5]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 512,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [6]:
# model = NERModel('bert', 
#                 'outputs/checkpoint-594-epoch-1',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# # result, model_outputs, wrong_preds = model.eval_model(eval_df)
# # result

In [7]:
model = NERModel('bert',
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Downloading: 100%|██████████| 0.99k/0.99k [00:00<00:00, 1.01MB/s]
Downloading: 100%|██████████| 424M/424M [00:22<00:00, 19.9MB/s] 
Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

In [8]:
model.train_model(train_df,  eval_data=eval_df)

2022-01-08 18:14:42 simpletransformers.ner.ner_model INFO:  Converting to features started.
Epoch 1 of 5:   0%|          | 0/5 [00:00<?, ?it/s]d:\ner-idn-tweet\env\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epochs 0/5. Running Loss:    0.2850: 100%|██████████| 594/594 [02:14<00:00,  4.40it/s]
2022-01-08 18:17:10 simpletransformers.ner.ner_model INFO:  Converting to features started.
Epochs 1/5. Running Loss:    0.4622: 100%|██████████| 594/594 [02:11<00:00,  4.52it/s]
2022-01-08 18:19:42 simpletransformers.ner.n

(2970,
 defaultdict(list,
             {'global_step': [594, 1188, 1782, 2000, 2376, 2970],
              'train_loss': [0.28500896692276,
               0.4622194170951843,
               0.16505146026611328,
               0.2888239324092865,
               0.12291739881038666,
               0.15691225230693817],
              'eval_loss': [0.37170331910152565,
               0.3270120592225318,
               0.3189407006506152,
               0.33661948376954004,
               0.32954958916310495,
               0.34754173933909643],
              'precision': [0.4911660777385159,
               0.5147324113048707,
               0.5187817258883248,
               0.5187660668380463,
               0.5348961108151306,
               0.5326530612244897],
              'recall': [0.30562884784520666,
               0.376429199648197,
               0.4494283201407212,
               0.44371152154793314,
               0.4415127528583993,
               0.45910290237467016],
       

In [9]:
result, model_outputs, y_pred = model.eval_model(test_df)

2022-01-08 18:27:23 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Evaluation: 100%|██████████| 186/186 [00:10<00:00, 17.03it/s]
2022-01-08 18:27:40 simpletransformers.ner.ner_model INFO: {'eval_loss': 0.22311790902868553, 'precision': 0.7097541633624108, 'recall': 0.6265313265663283, 'f1_score': 0.6655512176984569}


In [10]:
result

{'eval_loss': 0.22311790902868553,
 'precision': 0.7097541633624108,
 'recall': 0.6265313265663283,
 'f1_score': 0.6655512176984569}

In [11]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [14]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    #["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

2022-01-08 18:28:12 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 11.97it/s]


In [15]:
predictions

[[{'@OPPOIndonesia': 'O'},
  {'✍Butuh': 'O'},
  {'memori': 'O'},
  {'besar': 'O'},
  {'128GB': 'O'},
  {'di': 'O'},
  {'OPPO': 'B-PROD'},
  {'A54': 'I-PROD'},
  {'buat': 'O'},
  {'simpan': 'O'},
  {'file-file': 'O'},
  {'tugas': 'O'},
  {'kuliah': 'O'},
  {'dan': 'O'},
  {'testimoni': 'O'},
  {'online': 'O'},
  {'shop': 'O'},
  {'-ku': 'O'},
  {'��Semoga': 'O'},
  {'oppo': 'B-PROD'},
  {'A54': 'I-PROD'},
  {'bisa': 'O'},
  {'menunjang': 'O'},
  {'penghasilan': 'O'},
  {'keluargaku': 'O'},
  {'��': 'O'},
  {'@OPPOIndonesia': 'O'},
  {'@nisa_rkt': 'O'},
  {'@Bebheey': 'O'},
  {'@kus_ica': 'O'},
  {'@Erna19_': 'O'},
  {'@Saftriyuni': 'O'},
  {'#OPPOA54': 'O'},
  {'https://t.co/bMYIiRb4QF': 'O'},
  {'��': 'O'},
  {'2.921': 'O'}]]

In [16]:
#Now we create the model again using the best estimators

print(sklearn.metrics.classification_report(
    y_test, y_pred, labels=label_list, digits=3
    ))

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [17]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [18]:
results, results_per_tag = evaluator.evaluate()

In [19]:
results['partial']

{'correct': 1910,
 'incorrect': 0,
 'partial': 255,
 'missed': 692,
 'spurious': 357,
 'possible': 2857,
 'actual': 2522,
 'precision': 0.8078905630452022,
 'recall': 0.7131606580329016,
 'f1': 0.7575757575757576}

In [20]:
results['exact']

{'correct': 1910,
 'incorrect': 255,
 'partial': 0,
 'missed': 692,
 'spurious': 357,
 'possible': 2857,
 'actual': 2522,
 'precision': 0.7573354480570975,
 'recall': 0.6685334266713335,
 'f1': 0.7101691764268452}

In [21]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [22]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)

In [23]:
import re
for i in range(len(y_test)):
    if len(y_test[i]) != len(y_pred[i]):
        print(x_test[i])
        print(len(x_test[i]))
        print("y_test       ",len(y_test[i]),y_test[i])
        print("y_pred before",len(y_pred[i]),y_pred[i])
        for j in range(len(x_test[i])):
            # if '�' in x_test[i][j]: 
            qmark=re.match(r'^�+$',x_test[i][j])
            if qmark:
                print(qmark)
                y_pred[i].insert(j,'O')
        print("y_pred after ",len(y_pred[i]), y_pred[i])
        print()